In [1]:
from datosipc import datosIPC

In [2]:
p = datosIPC(2023, 1, dbBackup=True)

In [3]:
p.inflacion_CA_RD_MEX()


([('Pais', 'Ene-2022', 'Ene-2023'),
  ('Guatemala', 2.8682743333109384, 9.690479299986944),
  ('El Salvador', 6.489465730287147, 7.028967359545879),
  ('Honduras', 6.177056646740664, 8.927613941018775),
  ('Nicaragua', 7.68237303670376, 10.92196386427824),
  ('Costa Rica', 3.4949190457177037, 7.656922703406854),
  ('Republica Dominicana', 8.726424044628288, 7.24071133139037),
  ('Panamá', 1.885014964262699, 2.7478745321342757),
  ('México', 7.070138825877881, 7.8795274656361824)],
 'Para el mes de enero 2023, en Centro América, República Dominicana y México, Nicaragua presentó la mayor tasa de inflación interanual de 10.92%, mientras que Panamá registró la tasa más baja con un nivel de 2.75%.')

In [1]:
from sqline import sqlINE

In [2]:
p = sqlINE(2023, 2, dbBackup=1)

In [10]:
datos = p.serie_historica("interanual")
datos

[('Ene-2012', 5.493446573310612),
 ('Feb-2012', 5.220711707457415),
 ('Mar-2012', 4.648716348995419),
 ('Abr-2012', 4.2752072052612),
 ('May-2012', 3.898090077071803),
 ('Jun-2012', 3.4720810738378516),
 ('Jul-2012', 2.8624150293321327),
 ('Ago-2012', 2.708315549058904),
 ('Sep-2012', 3.273040500494351),
 ('Oct-2012', 3.3491101740794615),
 ('Nov-2012', 3.1122615392777497),
 ('Dic-2012', 3.4438009354084143),
 ('Ene-2013', 3.85521675561713),
 ('Feb-2013', 4.181254131172651),
 ('Mar-2013', 4.346207567921456),
 ('Abr-2013', 4.131317623640252),
 ('May-2013', 4.269815731276871),
 ('Jun-2013', 4.792666561001746),
 ('Jul-2013', 4.736416868759696),
 ('Ago-2013', 4.418096753665135),
 ('Sep-2013', 4.210974860126404),
 ('Oct-2013', 4.1522236383634725),
 ('Nov-2013', 4.627905218407036),
 ('Dic-2013', 4.386275715984933),
 ('Ene-2014', 4.148259630962925),
 ('Feb-2014', 3.5053431575849947),
 ('Mar-2014', 3.2462041687517207),
 ('Abr-2014', 3.26503668076783),
 ('May-2014', 3.225001872656974),
 ('Jun-201

In [11]:
max(datos, key=lambda x: x[1])

('Feb-2023', 9.920699999743832)

In [12]:
min(datos, key=lambda x: x[1])

('Feb-2020', 1.236029923138049)

In [15]:
import pandas as pd

In [25]:
df1 = pd.DataFrame(p.serie_historica("interanual"), columns=["Fecha", "interanual"])

In [26]:
df2 = pd.DataFrame(p.serie_historica("intermensual"), columns=["Fecha", "intermensual"])

In [30]:
df = pd.merge(df1, df2, on="Fecha", how="right").fillna("-")
df

,Fecha,interanual,intermensual
0,Ene-2011,-,0.761897
1,Feb-2011,-,0.764491
2,Mar-2011,-,0.868646
3,Abr-2011,-,0.735185
4,May-2011,-,0.497378
...,...,...,...
141,Oct-2022,9.707482,0.981715
142,Nov-2022,9.173047,-0.004906
143,Dic-2022,9.231762,0.228328
144,Ene-2023,9.685003,0.376934


In [31]:
df.to_latex("tabla_prueba.tex", index=False)

C:\Users\laalvarado\AppData\Local\Temp\ipykernel_15980\468587983.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex("tabla_prueba.tex", index=False)


In [32]:
with open('tabla_prueba.tex', 'w') as f:
    f.write(df.style.hide_index().set_caption('Tabla de prueba').set_table_styles([{'selector': 'caption', 'props': [('text-align', 'center')]}]).to_latex())

C:\Users\laalvarado\AppData\Local\Temp\ipykernel_15980\721903093.py:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  f.write(df.style.hide_index().set_caption('Tabla de prueba').set_table_styles([{'selector': 'caption', 'props': [('text-align', 'center')]}]).to_latex())


In [34]:
with open('tabla_prueba.tex', 'w') as f:
    f.write(df.to_latex(index=False, longtable=True, caption='Tabla de prueba', header=["Fecha", "Mensual", "Anual"], 
                        column_format='|c|c|c|', escape=False).replace('\\hline', '\\cline{1-3}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))

C:\Users\laalvarado\AppData\Local\Temp\ipykernel_15980\2776811672.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df.to_latex(index=False, longtable=True, caption='Tabla de prueba', header=["Fecha", "Mensual", "Anual"],


In [36]:
with open('tabla_prueba.tex', 'w') as f:
    f.write(df.to_latex(index=False, longtable=True, caption='Tabla de prueba', 
                        column_format='|c|c|c|', escape=False, 
                        header=['Fecha', 'Interanual', 'Intermensual']).replace('\\hline', '\\cline{1-3}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))

C:\Users\laalvarado\AppData\Local\Temp\ipykernel_15980\1659245302.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df.to_latex(index=False, longtable=True, caption='Tabla de prueba',


In [38]:
def export_to_longtable(df, filename, caption, column_format='|c|c|', header=True):
    with open(filename, 'w+') as f:
        if header:
            f.write(df.head(1).to_latex(index=False, longtable=False, column_format=column_format, escape=False, header=header).replace('\\hline', '\\cline{1-2}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))
        f.write(df.to_latex(index=False, longtable=True, column_format=column_format, escape=False, header=False).replace('\\hline', '\\cline{1-2}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))
        f.seek(0)
        contents = f.read()
        f.seek(0)
        f.write('\\begin{longtable}{' + column_format + '}\n')
        f.write('\\caption{' + caption + '} \\\\\n')
        f.write('\\hline\n')
        f.write(contents)
        f.write('\\end{longtable}\n')

    
export_to_longtable(df, 'tabla_prueba.tex', 'Tabla de prueba')

C:\Users\laalvarado\AppData\Local\Temp\ipykernel_15980\580505376.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df.head(1).to_latex(index=False, longtable=False, column_format=column_format, escape=False, header=header).replace('\\hline', '\\cline{1-2}').replace('\\endfirsthead', '\\hline\n\\endhead\n'))
C:\Users\laalvarado\AppData\Local\Temp\ipykernel_15980\580505376.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df.to_latex(index=False, longtable

In [39]:
def export_to_longtable(df, filename, caption, column_format='c'*len(df.columns), header=True):
    """
    Exporta un DataFrame a un archivo .tex en formato longtable.
    
    Parameters:
    df (pandas.DataFrame): DataFrame a exportar.
    filename (str): Nombre del archivo .tex a crear.
    caption (str): Título de la tabla.
    column_format (str, optional): Formato de las columnas. Por defecto es 'c'*len(df.columns).
    header (bool, optional): Si se incluye la primera fila como encabezado. Por defecto es True.
    """
    with open(filename, 'w') as f:
        f.write('\\begin{longtable}{' + column_format + '}\n')
        if header:
            f.write('\\caption{' + caption + '}\\label{' + filename[:-4] + '-label}\\tabularnewline\n')
            f.write('\\toprule\n')
            for col in df.columns:
                f.write('\\textbf{' + col + '}')
                if col != df.columns[-1]:
                    f.write(' & ')
                else:
                    f.write(' \\\\\n')
            f.write('\\midrule\n')
            f.write('\\endfirsthead\n')
            f.write('\\multicolumn{' + str(len(df.columns)) + '}{c}{{Tabla \\thetable{} continuación de la página anterior.}} \\\\\n')
            f.write('\\toprule\n')
            for col in df.columns:
                f.write('\\textbf{' + col + '}')
                if col != df.columns[-1]:
                    f.write(' & ')
                else:
                    f.write(' \\\\\n')
            f.write('\\midrule\n')
            f.write('\\endhead\n')
        else:
            f.write('\\caption{' + caption + '}\\label{' + filename[:-4] + '-label}\\tabularnewline\n')
            f.write('\\toprule\n')
        for i, row in df.iterrows():
            for j, val in enumerate(row):
                if pd.isna(val):
                    val = '-'
                if isinstance(val, float):
                    f.write('{:.6f}'.format(val))
                else:
                    f.write(str(val))
                if j != len(row) - 1:
                    f.write(' & ')
                else:
                    f.write(' \\\\\n')
            f.write('\\midrule\n')
        f.write('\\end{longtable}\n')


In [80]:
def export_to_longtable(df, filename, caption, column_format='c'*len(df.columns), header=True, decimals=2):
    """
    Exporta un dataframe a un longtable de LaTeX en un archivo.
    
    Parameters:
    df (pandas.DataFrame): DataFrame a exportar.
    filename (str): Nombre del archivo a guardar.
    caption (str): Leyenda de la tabla.
    column_format (str): Cadena que describe el formato de las columnas.
    header (bool): Si True, muestra el encabezado de la tabla.
    decimals (int): Cantidad de decimales para los números. Si el valor no es numérico, se muestra tal cual.
    """
    with open(filename, 'w', encoding="utf-8") as f:
        f.write('\\begin{longtable}{' + column_format + '}\n')
        f.write('\\caption{' + caption + '}\\label{tab:' + filename[:-4] + '}\\\\\n')
        f.write('\\toprule\n')
        
        # Escribir el encabezado
        if header:
            f.write(' & '.join([col.replace('_', '\\_') for col in df.columns]))
            f.write('\\\\\\midrule\n')
        
        f.write('\\endfirsthead\n')
        f.write('\\multicolumn{' + str(len(df.columns)) + '}{c}{{\\bfseries \\tablename\\ \\thetable{} -- '
                'Continuación de la página anterior}}\\\\\n')
        f.write('\\toprule\n')
        if header:
            f.write(' & '.join([col.replace('_', '\\_') for col in df.columns]))
            f.write('\\\\\\midrule\n')
        f.write('\\endhead\n')
        f.write('\\midrule\n')
        f.write('\\multicolumn{' + str(len(df.columns)) + '}{r}{{Continúa en la siguiente página}}\\\\\n')
        f.write('\\endfoot\n')
        f.write('\\bottomrule\n')
        f.write('\\endlastfoot\n')
        
        # Escribir los datos
        from math import copysign
        for _, row in df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append('')
                elif isinstance(value, (int, float)):
                    valor = round(value, decimals)
                    if str(valor).replace('.', '') == '0' * decimals:
                        valor = int(valor)
                    valor = '{:.{}f}'.format(valor, decimals)
                    values.append(valor)
                else:
                    values.append(str(value))
            f.write(' & '.join(values))
            f.write('\\\\\n')
        
        f.write('\\end{longtable}\n')


In [81]:
export_to_longtable(df, 'tabla_prueba2.tex', 'Tabla variación interanual e intermnesual', decimals=3) 
